# 最长上升子序列

最长上升子序列（Longest increasing subsequence）的算法还挺有趣的，在此记录。

## 题目描述

> 给你一个序列 $X$，输出这个序列中的最长上升子序列。

In [98]:
import random

X = list(range(1, 11))
random.shuffle(X)
print('X =', X)

X = [8, 3, 9, 6, 7, 1, 5, 4, 2, 10]


初看上去，可以搜索。

In [99]:
# find LIS that end with X[index]
def LIS(index):
    if index == 0:
        return [X[0]]
    result = []
    for j in range(index):
        if X[j] < X[index]:
            result = max([result, LIS(j)], key=len)
    result.append(X[index])
    return result

LIS = max([LIS(i) for i in range(len(X))], key=len)
print(LIS)

[3, 6, 7, 10]


实际操作上可以记忆化 `LIS` 函数，但是这也只能优化最后一行找所有结尾时的计算。时间复杂度 $\Theta (n^2)$.

我们反思一下，这个算法之所以慢，是因为它是从后往前递归的。后面一个 LIS 的结果依赖它前面所有 LIS 的值。这就不满足最优子结构。我们有没有什么办法，可以从前到后一把遍历到底，直接出结果呢？

有的。你可以直接扫一遍这个数组，无论当前这个数是不是大于你记录的 LIS 里面的最后一个数，你都把它放在 LIS 中第一个比它大的那个数的位置。

In [100]:
import bisect

print('X =', X)
print('true LIS:', LIS)

def maybe_LIS():
    lis = []
    for num in X:
        i = bisect.bisect_left(lis, num)
        if i != len(lis):
            lis[bisect.bisect_left(lis, num)] = num
        else:
            lis.append(num)
        print(lis)
    return lis

print('LIS length:', len(maybe_LIS()))

X = [8, 3, 9, 6, 7, 1, 5, 4, 2, 10]
true LIS: [3, 6, 7, 10]
[8]
[3]
[3, 9]
[3, 6]
[3, 6, 7]
[1, 6, 7]
[1, 5, 7]
[1, 4, 7]
[1, 2, 7]
[1, 2, 7, 10]
LIS length: 4


[bisect 库文档](https://docs.python.org/3/library/bisect.html)

？？？

显然，只要 $X$ 不是单调递增的，最后 LIS 数组就是不合要求的。但是一个有趣的事实是：这样做出来的 LIS 数组的长度是正确的。如果你只需要长度，那么你可以直接使用这样的方法。

为什么呢？

我们先来感性地认识一下，观察上面的输出结果，每次 LIS 数组长度增加，都是当前数字大于它前一个数字。那么我们倒着推回去，其实并没有违反先后顺序，只是前面的数字被换成了别的数字而已。每个数字被插/覆盖进第 $i$ 位的时候，我们能确定的，就是存在一个长为 $i$ 且以这个数结尾的上升子序列。

但是如果这样就分析完了的话，还没有深入到这个算法本质上的优势：最优子结构。

（下面讲解默认数组下标从 1 开始）

我们定义，$A_{i,l}$ 为在前 $i$ 个数字中，长为 $l$ 的上升子序列的末位的最小值（注意可能有多个长为 $l$ 的上升子序列，我们选末位最小的）。$L_i$ 是前 $i$ 个数字中 LIS 的长度。如果 $l > L_i$，则 $A_{i,l}=+\infty$.

二维数组 $A_{i,l}$ 的子数列记为 $A_{i}$. 这些数列有个很好的性质：设数列 $S_{i,l}$ 为前 $i$ 个数字中，长为 $l$ 的上升子序列中末位最小的那个子序列，我们有

$$
\forall j \leq l,\ A_{i,j} \leq S_{i,l,j}
$$

利用反证法证明，假设存在 $j$ 使得 $A_{i,j} > S_{i,l,j}$，由于上升序列的子序列也上升，序列 $\{S_{i,l,1}, \cdots , S_{i,l,j}\}$ 的末位比 $A_{i,j}$ 更小，与 $A_{i,j}$ 为末位最小值相矛盾。证毕。

由这个性质，易得数列 $A_{i}$ 也是上升（单调递增）的。

这里用 loop invariant 证明，不失严谨性：

> 在每个 for 循环开始前，数列 $A_{i-1}$ 中的数都已被正确求出。

**初始化**：第一次 for 循环执行前 $i-1=0$，所以没有任何数字可被用来组成序列，即 $A_{0, l}=+\infty$. （Python 代码中 `A[-1]` 指倒数第一行，正好全为 $+\infty$.）

**维护**：每次循环时我们复制数列 $A_{i-1}$ 到 $A_i$，找出单调递增数列 $A_{i}$ 中第一个大于 $x_i$ 的数，并把它替换成 $x_i$. 显然如果 $A_{i,l} \neq x_i$，那么 $A_{i,l} = A_{i-1,l}$. 所以我们要找的就是使 $A_{i,l} = x_i$ 的 $l$. 由上述性质得

$$
\begin{gather*}
A_{i,l-1} \leq S_{i,l,l-1} < S_{i,l,l} = x_i \\
A_{i,l} < A_{i,l+1}
\end{gather*}
$$

即

$$
A_{i,l-1} < x_i < A_{i,l+1}
$$

这就证明了 $A_{i-1}$ 中第一个大于 $x_i$ 的数的下标就是我们要找的 $l$.

**终止**：循环到 $i=len(X)+1$ 时终止。此时 $A_{i-1}$ 中最后一个不为 $+\infty$ 的数的下标即为 LIS 长度。

In [106]:
A = [[999]*len(X) for i in range(len(X))]

for i in range(len(X)):
    A[i] = A[i-1].copy()
    A[i][bisect.bisect_left(A[i-1], X[i])] = X[i]
    print(A[i])

[8, 999, 999, 999, 999, 999, 999, 999, 999, 999]
[3, 999, 999, 999, 999, 999, 999, 999, 999, 999]
[3, 9, 999, 999, 999, 999, 999, 999, 999, 999]
[3, 6, 999, 999, 999, 999, 999, 999, 999, 999]
[3, 6, 7, 999, 999, 999, 999, 999, 999, 999]
[1, 6, 7, 999, 999, 999, 999, 999, 999, 999]
[1, 5, 7, 999, 999, 999, 999, 999, 999, 999]
[1, 4, 7, 999, 999, 999, 999, 999, 999, 999]
[1, 2, 7, 999, 999, 999, 999, 999, 999, 999]
[1, 2, 7, 10, 999, 999, 999, 999, 999, 999]
